<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_11_CNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Lecture_BigData'

# 学習済みモデルで推論

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input, decode_predictions

import numpy as np
import os, json, csv

image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [image.load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([image.img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return(output)

""" モデルの読み込み """
model = ResNet50(weights='imagenet') # <--- 別のモデルで検証できる
# model.load_weights(model_filename) # <--- 学習済みモデルの重み


""" テストデータの読み込み """
image_paths = 'dataset/test-images'
image_paths = [os.path.join(image_paths,filename) for filename in sorted(os.listdir(image_paths))]
test_data = read_and_prep_images(image_paths)

""" 推論 """
preds = model.predict(test_data)

""" Top3 書き込み """
#predicted = decode_predictions(preds, top=3, class_list_path='dataset/imagenet_class_index.json')
predicted = decode_predictions(preds, top=3)

with open("results.csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(["IMAGE_PATH", "TOP1", "TOP2", "TOP3"])
  for i, labels in enumerate(predicted):
    row = [image_paths[i], labels[0][1], labels[1][1], labels[2][1]]
    writer.writerow(row)
    print(row)

# 物体検出

In [ ]:
import IPython
from google.colab import output
from google.colab.patches import cv2_imshow # incompatible with Jupyter notebook
import cv2
import numpy as np
from PIL import Image, ImageDraw
from io import BytesIO
import base64
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input, decode_predictions
from keras.preprocessing import image

model = DenseNet121(weights='imagenet')

def run(img_str):
    #decode to image
    decimg = base64.b64decode(img_str.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8); 
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

    #############your process###############
    # DenseNet121画像判定
    resize_frame = cv2.resize(decimg, (300, 224))           # 640x480 -> 300x224(4:3)に画像リサイズ
    trim_x, trim_y = int((300-224)/2), 0                    # 判定用に224x224へトリミング
    trim_h, trim_w = 224, 224
    trim_frame = resize_frame[trim_y : (trim_y + trim_h), trim_x : (trim_x + trim_w)]
    x = image.img_to_array(trim_frame)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)                                # 画像AI判定

    # 認識物体名, 認識率 表示用
    TXT1 = "[DenseNet121 predict]"
    TXT2 = "TOP1 :" + decode_predictions(preds, top=3)[0][0][1] + " : " + str(int(decode_predictions(preds, top=3)[0][0][2] * 100)) + "%"
    TXT3 = "TOP2 :" + decode_predictions(preds, top=3)[0][1][1] + " : " + str(int(decode_predictions(preds, top=3)[0][1][2] * 100)) + "%"
    TXT4 = "TOP3 :" + decode_predictions(preds, top=3)[0][2][1] + " : " + str(int(decode_predictions(preds, top=3)[0][2][2] * 100)) + "%"

    img = Image.fromarray(trim_frame)
    draw = ImageDraw.Draw(img)
    draw.text((15, 10), TXT1, fill=(255, 255, 255, 0))
    draw.text((15, 30), TXT2, fill=(255, 255, 255, 0))
    draw.text((15, 50), TXT3, fill=(255, 255, 255, 0))
    draw.text((15, 70), TXT4, fill=(255, 255, 255, 0))
    img = np.array(img)
#   cv2_imshow(img)
    out_img = cv2.resize(img, (224*3, 224*3))               # 拡大表示
    #############your process###############

    #encode to string
    _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = encimg.tostring()
    img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
    return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

In [11]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

def use_cam(quality=0.8):
  js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      display_size = 640 
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size * video.videoHeight / video.videoWidth;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_num = 0
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);     
            if (send_num<1){
                send_num += 1
                const img = src_canvas.toDataURL('image/jpeg', quality);
                const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
                result.then(function(value) {
                    parse = JSON.parse(JSON.stringify(value))["data"]
                    parse = JSON.parse(JSON.stringify(parse))["application/json"]
                    parse = JSON.parse(JSON.stringify(parse))["img_str"]
                    var image = new Image()
                    image.src = parse;
                    image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                    send_num -= 1
                })
            }
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);   
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
  display(js)
  data = eval_js('useCam({})'.format(quality))

In [ ]:
use_cam()

# YOLO v5 モデル

In [ ]:
WORKING_DIR = '/content/tmp'
!mkdir $WORKING_DIR
%cd $WORKING_DIR

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5/
!pip install -qr requirements.txt

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip
!unzip coco128.zip

In [ ]:
!python train.py --img 640 --batch 16 --epochs 300 --data coco128.yaml --weights yolov5x.pt

In [ ]:
!python detect.py --source data/images --weights yolov5x.pt --conf 0.50

In [ ]:
import glob
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 20))
images = glob.glob("runs/detect/exp8/*.jpg")
for i in range(len(images)):
  ax = plt.subplot(3, 3, i + 1)
  img = image.load_img(images[i])
  plt.imshow(img)
  plt.axis("off")